### Dataset to use - change this value to analyze a different data set

In [1]:
dataset = 'twenty_small_world_new'

In [2]:
from analysis import check_folder
check_folder.check_folder(dataset)

['/home/ubuntu/active_learning/new/data/twenty_small_world_new/dags/dag0/entropy-enum,n=256,b=2,k=2/final_dags',
 '/home/ubuntu/active_learning/new/data/twenty_small_world_new/dags/dag0/entropy-enum,n=256,b=2,k=2/samples',
 '/home/ubuntu/active_learning/new/data/twenty_small_world_new/dags/dag1/entropy-enum,n=256,b=2,k=2/final_dags',
 '/home/ubuntu/active_learning/new/data/twenty_small_world_new/dags/dag1/entropy-enum,n=256,b=2,k=2/samples',
 '/home/ubuntu/active_learning/new/data/twenty_small_world_new/dags/dag10/entropy-enum,n=256,b=2,k=2/final_dags',
 '/home/ubuntu/active_learning/new/data/twenty_small_world_new/dags/dag10/entropy-enum,n=256,b=2,k=2/samples',
 '/home/ubuntu/active_learning/new/data/twenty_small_world_new/dags/dag11/entropy-enum,n=256,b=2,k=2/final_dags',
 '/home/ubuntu/active_learning/new/data/twenty_small_world_new/dags/dag11/entropy-enum,n=256,b=2,k=2/samples',
 '/home/ubuntu/active_learning/new/data/twenty_small_world_new/dags/dag12/entropy-enum,n=256,b=2,k=2/fin

In [3]:
import pandas as pd

def order_tpr_fpr(fprs, tprs):
    df = pd.DataFrame(dict(fpr=fprs, tpr=tprs))
    df.sort_values('fpr', ascending=False)
    df.drop_duplicates(subset='fpr', keep='last', inplace=True)
    return df
    
    
a = order_tpr_fpr([.2, .1, .2], [.3, .2, .4])

### Load in parent probabilities and rates

In [4]:
from analysis import check_gies, check_samples
import numpy as np
import itertools as itr

In [5]:
dag_folders = check_gies.get_dag_folders(dataset)
true_dags = check_gies.get_true_dags(dag_folders)
print('loaded DAGs')
parent_probs_by_dag = check_gies.get_parent_probs_by_dag(dag_folders, 9)
print('Computed parent probabilities')

loaded DAGs
/home/ubuntu/active_learning/new/data/twenty_small_world_new/dags/dag23


FileNotFoundError: [Errno 2] No such file or directory: '/home/ubuntu/active_learning/new/data/twenty_small_world_new/dags/dag23/entropy-enum,n=256,b=2,k=2/final_dags'

### Load the rates DataArray, a 7-dimensional tensor mapping each strategy (strategy name, n, b, k) and significance level to the value of a rate for all DAGs

In [ ]:
strategy_names = ['entropy', 'random', 'entropy-enum']
ks = [1, 2]
bs = [1, 2]
ns = [32, 256, 1024]
np.linspace(0, 1, 11)

In [ ]:
counts_da = check_samples.count_samples(dataset, strategy_names, ks, bs, ns)

In [ ]:
rates_da = check_gies.get_rates_data_array(
    parent_probs_by_dag,
    true_dags,
    target=9,
    strategy_names=strategy_names,
    ks=ks,
    bs=bs,
    ns=ns,
    alphas=np.linspace(0, 1, 11)
)
print(rates_da.dims)

### Plot curves for each strategy

In [ ]:
import matplotlib.pylab as plt
import seaborn as sns
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
%matplotlib inline
plt.rcParams["figure.figsize"] =(20,12)
sns.set(font_scale=1.5)

In [ ]:

linestyles = ['solid', 'dashed', 'dotted']
colors = sns.color_palette()
sample_handles = [
    mlines.Line2D([0], [0], color='k', linestyle=linestyle, label=n) 
    for n, linestyle in zip(ns, linestyles)
]
strat_handles = [
    mpatches.Patch(facecolor=color, label=strat)
    for strat, color in zip(strategy_names, colors)
]

In [ ]:
plt.clf()

rate_avgs_da = rates_da.mean(dim='dag')
b=1
k=1
for strategy, color in zip(strategy_names, colors):
    for n, linestyle in zip(ns, linestyles):
        strat_avg_tprs = rate_avgs_da.sel(strategy=strategy, b=b, k=k, n=n, rate='tpr').values
        strat_avg_fprs = rate_avgs_da.sel(strategy=strategy, b=b, k=k, n=n, rate='fpr').values
        tpr_fpr_df = order_tpr_fpr(strat_avg_fprs, strat_avg_tprs)
        plt.plot(tpr_fpr_df['fpr'], tpr_fpr_df['tpr'], linestyle=linestyle, color=color)

plt.legend(
    handles=strat_handles + sample_handles
)
plt.title(dataset + ', batches=%s, k=%s' % (b, k))
plt.xlabel('Average FPR')
plt.ylabel('Average TPR');

In [ ]:
fig, ax = plt.subplots(len(bs), len(ns), sharey=True, sharex=True)
k=2
for (b_ix, b), (n_ix, n) in itr.product(list(enumerate(bs)), list(enumerate(ns))):
    for strategy, color in zip(strategy_names, colors):
        avg_rates = rate_avgs_da.sel(strategy=strategy, b=b, k=k, n=n)
        tpr_fpr_df = order_tpr_fpr(avg_rates.sel(rate='fpr').values, avg_rates.sel(rate='tpr').values)
        ax[b_ix, n_ix].plot(tpr_fpr_df['fpr'], tpr_fpr_df['tpr'], color=color)
        if b_ix == len(bs)-1:
            ax[b_ix, n_ix].set_xlabel('n = %s' % n)
        if n_ix == 0:
            ax[b_ix, n_ix].set_ylabel('b = %s' % b)


In [ ]:
from scipy.stats import entropy

In [ ]:
n = 256
k = 2
b = 2

c_e = counts_da.sel(strategy='entropy', k=k, b=b, n=n)/n
c_e

In [ ]:
c_r = counts_da.sel(strategy='random', k=k, b=b, n=n)/n

In [ ]:
ent_e = entropy(c_e.T)
ent_r = entropy(c_r.T)

In [ ]:
ent_e

In [ ]:
ent_e.mean()

In [ ]:
ent_r

In [ ]:
ent_r.mean()

In [ ]:
np.log(4)

In [ ]:
import random
from collections import Counter

def random_choices(p, k):
    c = Counter(random.choices(list(range(p)), k=k))
    arr = np.zeros(p)
    for i, val in c.items():
        arr[i] += val
    return arr/arr.sum()


In [ ]:
for k in [20, 40, 100, 200]:
    print(entropy(random_choices(20, k)))

In [ ]:
entropy([.5, .5])

In [ ]:
np.log(2)

In [ ]:
c_e

In [ ]:
counts_da.sel(strategy='entropy', k=1, b=1, n=256, dag=1)

In [ ]:
counts_da.sel(strategy='random', k=2, b=2, n=256, dag=1)